#### CELL-1 코드리뷰 

- <strong>NAME = ''</strong> -> 가져올 모델의 이름을 입력합니다 *전 과정에서 지정했던 이름으로 입력합니다


In [ ]:
# Set Hyper Parameters
NAME = '' #모델의 이름을 입력합니다 예시: '12'

<br></br>

#### CELL-2 코드리뷰 

- 사용했던 코드를 통합하여 주행을 진행합니다


In [ ]:
import jajucha2
import torch
import numpy as np
import torchvision.transforms as transforms
from PIL import Image as Img
import cv2

#인공지능 모델 가져오기
device = torch.device('cuda')
model = jajucha2.ai.get_model(NAME)
model.to(device)
model.eval()

steer = 0
speed = 0

while True:
    image = jajucha2.camera.get_image('center')

    #그리드 기반 주행 코드
    (V,L,R) ,grid = jajucha2.camera.gridFront(image)

    if(V[5] < 100):
        steer = -10
        speed = 5
    else:
        steer = 0
        speed = 5

    #깊이기반 거리감지 주행
    depth = jajucha2.camera.get_depth() 
    jajucha2.camera.show_image(depth, 'depth')  
    height, width = depth.shape[:2]
    center_x, center_y = width // 2, height // 2
    region_size = 60
    start_x = center_x - region_size // 2  
    start_y = center_y - region_size // 2  
    center_region = depth[start_y:start_y + region_size, start_x:start_x + region_size]
    mean_value = np.mean(center_region)
    if(mean_value > 100):
        speed = 0

    ## 인공지능 기반 신호등 감지
    image = Img.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB)) 
    image = transform(image)  
    image =image.unsqueeze(0).to(device)
    output = (model(image))
    max_index = torch.argmax(output)
    if(max_index == 1):
        speed = 0
    
    jajucha2.control.set_motor(steer,steer,speed)




